# Scanner & Messenger Agents and Pushover Integration

# Libraries Import

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from Agents_Folder.Deal_Scraper import ScrapedDeal, DealSelection
import logging
import requests
load_dotenv(override=True)
openai = OpenAI()
MODEL = 'gpt-5.2'

ModuleNotFoundError: No module named 'Agents_Folder.deals'

In [9]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [01:47<00:00, 21.55s/it]


In [10]:
len(deals)

50

# GPT-5.2 is used to summarise deals and identify their prices

In [11]:
SYSTEM_PROMPT = """You identify and summarize the 10 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 10 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this is not the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 10 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 10 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this is not the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 10 deals, no more."

In [15]:
def update_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [17]:

user_prompt = update_user_prompt(deals)
print(user_prompt[:3000])
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]

Respond with the most promising 10 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 10 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this is not the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Beats Solo Buds for $39 + free shipping
Details: Use coupon code "VIPOUTLET2026" to drop it to the lowest price we've ever seen. Buy Now at eBay
Features: Dolby Atmos Bluetooth 18 hours Model: MUW03LL/A
URL: https://www.dealnews.com/products/Beats/Beats-Solo-Buds/484671.html?iref=rss-c142

Title: VIP Outlet Deals at eBay: Up to 30% off + extra 10% off + free shipping
Details

In [21]:
response = openai.chat.completions.parse(model=MODEL, messages=messages, response_format=DealSelection, reasoning_effort="none")
results = response.choices[0].message.parsed
results

DealSelection(deals=[Deal(product_description='Beats Solo Buds are true wireless Bluetooth earbuds designed for everyday listening with support for Dolby Atmos spatial audio. They’re rated for up to 18 hours of battery life, making them suitable for commutes and long sessions between charges. Model MUW03LL/A.', price=39.0, url='https://www.dealnews.com/products/Beats/Beats-Solo-Buds/484671.html?iref=rss-c142'), Deal(product_description='The Senbono H89 is a budget-friendly smartwatch that pairs with your phone to provide basic smart-watch functionality such as notifications and activity-style tracking (features vary by app/device). Its lightweight, wearable design is intended for daily use and casual fitness monitoring. It’s sold as a standalone watch with included charging method depending on the listing.', price=20.0, url='https://www.dealnews.com/Senbono-H89-Smart-Watch-for-20-free-shipping/21803549.html?iref=rss-c142'), Deal(product_description='Apple iPhone 17 Air (unlocked) is a 

In [22]:
for deal in results.deals:
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print()


Beats Solo Buds are true wireless Bluetooth earbuds designed for everyday listening with support for Dolby Atmos spatial audio. They’re rated for up to 18 hours of battery life, making them suitable for commutes and long sessions between charges. Model MUW03LL/A.
39.0
https://www.dealnews.com/products/Beats/Beats-Solo-Buds/484671.html?iref=rss-c142

The Senbono H89 is a budget-friendly smartwatch that pairs with your phone to provide basic smart-watch functionality such as notifications and activity-style tracking (features vary by app/device). Its lightweight, wearable design is intended for daily use and casual fitness monitoring. It’s sold as a standalone watch with included charging method depending on the listing.
20.0
https://www.dealnews.com/Senbono-H89-Smart-Watch-for-20-free-shipping/21803549.html?iref=rss-c142

Apple iPhone 17 Air (unlocked) is a 5G smartphone with 256GB of built-in storage for photos, apps, and video. Being unlocked, it’s designed to work across compatible c

In [23]:
root = logging.getLogger()
root.setLevel(logging.INFO)

# ScannerAgent Call

In [27]:
from agents.scanner_agent import ScannerAgent
scanner_agent = ScannerAgent()
scanner_results = scanner_agent.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 50 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Outputs
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


In [28]:
scanner_results

DealSelection(deals=[Deal(product_description='Beats Solo Buds are compact true wireless earbuds with Dolby Atmos support for immersive spatial audio and Bluetooth connectivity. They promise up to 18 hours of combined playback time with the charging case and a slim, on‑ear charging and control design (Model MUW03LL/A). These are aimed at users who want portable, branded earbuds with enhanced audio processing for movies and music.', price=39.0, url='https://www.dealnews.com/products/Beats/Beats-Solo-Buds/484671.html?iref=rss-c142'), Deal(product_description='LG G5 Series OLED65G5WUA is a 65" 4K OLED smart TV with 3840x2160 resolution and a 120Hz refresh rate, delivering deep blacks and wide color via OLED technology. It supports Dolby Vision, HDR10 and HLG, runs webOS 25 with Alexa and Google Assistant compatibility, and offers multiple inputs including 4 HDMI and 3 USB ports plus optical audio and Ethernet. The set includes features for gamers and home theater setups and comes certifie

# Pushover

In [29]:
load_dotenv(override=True)
pushover_user_key = os.getenv('PUSHOVER_USER')
pushover_application_key = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"

In [30]:
if pushover_user_key:
    print(f"Pushover user found and starts with {pushover_user_key[0]}")
else:
    print("Pushover user not found")

if pushover_application_key:
    print(f"Pushover token found and starts with {pushover_application_key[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


# Push function to send a notification to users

In [31]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user_key, "token": pushover_application_key, "message": message}
    requests.post(pushover_url, data=payload)

# Agent Test

In [32]:
from agents.messaging_agent import MessagingAgent

agent = MessagingAgent()
agent.push("SUCH A MASSIVE DEAL!!")

INFO:root:[Messaging Agent] Messaging Agent is initializing
INFO:root:[Messaging Agent] Messaging Agent has initialized Pushover and GPT-5-MINI
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification


In [ ]:
agent.notify("A special deal on Sumsung 60 inch LED TV going at a great bargain", 300, 1000, "www.samsung.com") # Description, reduced amount, predicted value, and URL

INFO:root:[Messaging Agent] Messaging Agent is using GPT-5-MINI to craft the message
01:08:00 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
01:08:06 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification
INFO:root:[Messaging Agent] Messaging Agent has completed
